In [179]:
import pandas as pd
import json
import numpy as np

# Load the CSV file
csv_file_path = './data/prolific_export_65566952c6b6b608da9084b0.csv'
csv_data = pd.read_csv(csv_file_path)

# Load the JSON file
json_file_path = './data/user-study-narrative-sketch-default-rtdb-export.json'
with open(json_file_path, 'r') as file:
    actions = json.load(file)

# Extracting and renaming demographic information from the CSV data
# demographics = csv_data[['Participant id', 'Age', 'Sex', 'Ethnicity simplified', 'Country of birth', 'Country of residence', 'Nationality', 'Language', 'Student status', 'Employment status']]
demographics = csv_data[['Participant id', 'Age', 'Sex']]
demographics = demographics.rename(columns={
    'Participant id': 'Participant ID'
})


In [180]:

# Flatten the JSON data and combine with demographic data
flattened_data = []
for participant_id, data in actions.items():
    # Ensure participant is in demographics
    if participant_id not in demographics['Participant ID'].values:
        continue

    demo_info = demographics[demographics['Participant ID'] == participant_id].iloc[0].to_dict()

    # Route Path Navigation Data
    for route_data in data.get('routePathNavigation', {}).values():
        combined_data = {**demo_info, **route_data, 'navType': 'routePathNav'}
        flattened_data.append(combined_data)

    # User Selections Data
    for selection_data in data.get('userSelections', {}).values():
        combined_data = {**demo_info, **selection_data, 'navType': 'userSelectionNav'}
        flattened_data.append(combined_data)

# Convert to DataFrame
combined_data_df = pd.DataFrame(flattened_data)

In [181]:
combined_data_df.isnull().sum()

Participant ID      0
Age                 0
Sex                 0
routePath         103
timestamp           0
navType             0
currentIndex       18
decision           18
mode               18
reason             18
rightSelection     18
selection          18
story              18
dtype: int64

In [182]:
print(combined_data_df.head(1))
print(combined_data_df.dtypes)


             Participant ID Age     Sex routePath      timestamp  \
0  5d05c531dc2d54001838ce76  44  Female    /intro  1700167160650   

        navType  currentIndex decision mode reason rightSelection selection  \
0  routePathNav           NaN      NaN  NaN    NaN            NaN       NaN   

  story  
0   NaN  
Participant ID     object
Age                object
Sex                object
routePath          object
timestamp           int64
navType            object
currentIndex      float64
decision           object
mode               object
reason             object
rightSelection     object
selection          object
story              object
dtype: object


In [183]:
import datetime
import pytz

def convert_timestamp_to_australia_datetime(timestamp, timezone='Australia/Sydney'):
    # The timestamp is in milliseconds, convert it to seconds
    timestamp_in_seconds = timestamp / 1000.0

    # Convert to datetime
    dt_utc = datetime.datetime.utcfromtimestamp(timestamp_in_seconds)

    # Set the timezone to UTC
    dt_utc = dt_utc.replace(tzinfo=pytz.UTC)

    # Convert to the desired timezone
    tz_australia = pytz.timezone(timezone)
    dt_australia = dt_utc.astimezone(tz_australia)

    return dt_australia



In [188]:
# Convert the timestamp (int64) column to datetime
combined_data_df['time'] = combined_data_df['timestamp'].apply(convert_timestamp_to_australia_datetime)

# Convert the currentIndex column to integer
combined_data_df['index'] = combined_data_df['currentIndex'].fillna(0).astype(int)

# Updating the Route Path for 'training' and 'task' modes using vectorized operations
condition = combined_data_df['mode'].isin(['training', 'task'])
combined_data_df['route'] = np.where(
    condition, 
    "/" + combined_data_df['mode'] + "/" + combined_data_df['index'].astype(str),
    combined_data_df['routePath']
)
# Dropping old timestamp columns and other unnecessary columns
# combined_data_df = combined_data_df.drop(columns=['timestamp', 'mode', 'currentIndex', 'routePath'])

# Sort the data by participant ID and time
combined_data_df = combined_data_df.sort_values(by=['Participant ID', 'time'])

# get the time difference as duration, it should be in seconds, group by participant ID, next time minus current time
combined_data_df['duration'] = combined_data_df.groupby('Participant ID')['time'].shift(-1) - combined_data_df['time']

combined_data_df['duration'] = combined_data_df['duration'].dt.total_seconds()

combined_data_df


,Participant ID,Age,Sex,routePath,timestamp,navType,currentIndex,decision,mode,reason,rightSelection,selection,story,time,index,route,duration
0,5d05c531dc2d54001838ce76,44,Female,/intro,1700167160650,routePathNav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-17 07:39:20.650000+11:00,0,/intro,97.742
1,5d05c531dc2d54001838ce76,44,Female,/motifs,1700167258392,routePathNav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-17 07:40:58.392000+11:00,0,/motifs,40.471
2,5d05c531dc2d54001838ce76,44,Female,/about,1700167298863,routePathNav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-17 07:41:38.863000+11:00,0,/about,29.508
3,5d05c531dc2d54001838ce76,44,Female,/training,1700167328371,routePathNav,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-11-17 07:42:08.371000+11:00,0,/training,72.928
6,5d05c531dc2d54001838ce76,44,Female,NaN,1700167401299,userSelectionNav,0.0,cancel,training,,ShortFork,Linear,story_Muddle_Puddle_ShortFork_seed942.json,2023-11-17 07:43:21.299000+11:00,0,/training/0,6.060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,603b750eab9d377aeac41724,25,Female,NaN,1700167793622,userSelectionNav,6.0,confirm,task,last passage is a bit random and disrupts the ...,Ladder,LongFork,story_Muddle_Puddle_Ladder_seed241.json,2023-11-17 07:49:53.622000+11:00,6,/task/6,46.442
118,603b750eab9d377aeac41724,25,Female,NaN,1700167840064,userSelectionNav,7.0,confirm,task,The story is relevant and follows on from each...,LongFork,Ladder,story_Muddle_Puddle_LongFork_seed288.json,2023-11-17 07:50:40.064000+11:00,7,/task/7,48.656
119,603b750eab9d377aeac41724,25,Female,NaN,1700167888720,userSelectionNav,8.0,confirm,task,"Three separate passages. However, the first an...",WideBranch,WideMerge,story_Muddle_Puddle_WideBranch_seed496.json,2023-11-17 07:51:28.720000+11:00,8,/task/8,39.981
120,603b750eab9d377aeac41724,25,Female,NaN,1700167928701,userSelectionNav,9.0,confirm,task,First and last passage seem to link together.,SharpBranch,WideMerge,story_Muddle_Puddle_SharpBranch_seed178.json,2023-11-17 07:52:08.701000+11:00,9,/task/9,0.054


In [185]:
combined_data_df.to_csv('output.csv', index=False)